# Import NumPy and Check Python Version

In [1]:
import sys
import numpy as np

print('You\'re running python %s' % sys.version.split(' ')[0])

You're running python 3.8.9


<h2>Euclidean Distances in Python</h2>

<p>Many machine learning algorithms access their input data primarily through pairwise (Euclidean) distances, therefore it is important that we have a fast function that computes pairwise distances of input vectors.</p>

<p>Assume we have $n$ data vectors $\mathbf{x_1},\dots,\mathbf{x_n}\in{\cal R}^d$ and $m$ vectors $\mathbf{z_1},\dots,z_m\in{\cal R}^d$. With these data vectors, let us define two matrices $X=[\mathbf{x_1},\dots,\mathbf{x_n}]\in{\cal R}^{n\times d}$, where the $i^{th}$ row is a vector $\mathbf{x_i}$ and similarly $Z=[\mathbf{z_1},\dots,\mathbf{z_m}]\in{\cal R}^{m\times d}$.</p>

<p>We want a distance function that takes as input these two matrices $X$ and $Z$ and outputs a matrix $D\in{\cal R}^{n\times m}$, where $$D_{ij}=\sqrt{(\mathbf{x_i}-\mathbf{z_j})(\mathbf{x_i}-\mathbf{z_j})^\top}.$$</p>

In [2]:
def l2distanceSlow(X, Z=None):
    if Z is None:
        Z = X
    
    n, d = X.shape
    m = Z.shape[0]
    
    # allocate memory for the output matrix
    D = np.zeros((n,m))
    for i in range(n):     # loop over vectors in X
        for j in range(m): # loop over vectors in Z
            D[i,j] = 0.0
            for k in range(d): # loop over dimensions
                # compute l2-distance between the ith and jth vector
                D[i,j] = D[i,j] + (X[i,k] - Z[j,k]) ** 2;
            D[i,j] = np.sqrt(D[i,j]); # take square root
            
    return D

In [3]:
X = np.random.rand(700, 100)
print('Running the naive version for the...')
%time Dslow = l2distanceSlow(X)

Running the naive version for the...
Wall time: 56.4 s


This code defines some random data in $X$ and computes the corresponding distance matrix $D$. The <em>%time</em> statement determines how long this code takes to run. This implementation is much too slow for such a simple operation on a small amount of data, and writing code like this to deal with matrices will result in code that takes <strong>days</strong> to run.

<strong>As a general rule, we should avoid tight loops at all cost.</strong> We can do much better by performing bulk matrix operations using the NumPy package, which calls highly optimized compiled code behind the scenes.

<h2>Efficient Programming with NumPy</h2>

<p>Although there is an execution overhead per line in Python, matrix operations are optimized and fast. Python for scientific computing can be very fast if almost all the time is spent on a few heavy duty matrix operations. In this exercise, we will transform the function above into a few matrix operations <em>without any loops at all.</em></p>

<p>The key to efficient programming in Python for machine learning in general is to think about it in terms of mathematics and not in terms of loops.</p>

<h2>Exercises</h2>

<p>The following three exercises will implement the euclidean distance function without loops.</p>

<h3>Exercise 1: Inner-Product Matrix</h3>

<p>Show that the Inner-Product Matrix (Gram matrix) can be expressed in terms of pure matrix multiplication.

$$G_{ij}=\mathbf{x}_i\mathbf{z}_j^\top$$

In [4]:
def innerproduct(X, Z=None):
    """
    This function computes the inner-product matrix.
    
    Syntax:
    ------
    D = innerproduct(X,Z)
    
    Inputs:
    ------
    X : nxd data matrix with n vectors (rows) of dimensionality d
    Z : mxd data matrix with m vectors (rows) of dimensionality d
    
    Outputs:
    -------
    Matrix G of size nxm
    G[i,j] is the inner-product between vectors X[i,:] and Z[j,:]
    
    This function can be called with only one input:
    innerproduct(X) = innerproduct(X,X)
    """
    if Z is None:
        Z = X
    res = np.dot(X, Z.T)
    
    return res

<h3>Exercise 2: Derive the Distance Matrix</h3>

Let us define two new matrices $S,R\in{\cal R}^{n\times m}$ 
$$S_{ij}=\mathbf{x}_i\mathbf{x}_i^\top, \ \ R_{ij}=\mathbf{z}_j\mathbf{z}_j^\top.$$
Show that the <em>squared</em>-euclidean matrix $D^2\in{\cal R}^{n\times m}$, defined as
$$D^2_{ij}=(\mathbf{x}_i-\mathbf{z}_j)(\mathbf{x}_i-\mathbf{z}_j)^\top,$$
can be expressed as a linear combination of the matrix $S, G, R$.</p></td>

<h3>Exercise 3: Implement <code>l2distance</code></h3>

<p>Implement the function <strong><code>l2distance</code></strong>, which computes the Euclidean distance matrix $D$ without a single loop.</p>

In [5]:
def l2distance(X, Z=None):
    """
    function D=l2distance(X,Z)
    
    This function computes the Euclidean distance matrix.
    
    Syntax:
    ------
    D = l2distance(X,Z)
    
    Inputs:
    ------
    X : nxd data matrix with n vectors (rows) of dimensionality d
    Z : mxd data matrix with m vectors (rows) of dimensionality d
    
    Outputs:
    -------
    Matrix D of size nxm
    D(i,j) is the Euclidean distance of X(i,:) and Z(j,:)
    
    This function can be called with only one input:
    l2distance(X) = l2distance(X,X)
    """
    if Z is None:
        Z = X
    
    n, d1 = X.shape
    m, d2 = Z.shape
    assert (d1 == d2), 'Dimensions of input vectors must match!'
    
    # matrix linear combination
    D = -2 * innerproduct(X,Z)
    S = np.expand_dims(np.sum(np.power(X,2), axis=1),1)
    R = np.expand_dims(np.sum(np.power(Z,2), axis=1),1)
    
    res = S + R.T + D
    res = np.maximum(res, 0)
    res = np.sqrt(res)
    
    return res

Let's now compare the speed of l2-distance function against the previous naïve implementation:

In [6]:
import time
current_time = lambda: int(round(time.time() * 1000))

X = np.random.rand(700,100)
Z = np.random.rand(300,100)

print('Running the naïve version...')
before = current_time()
Dslow = l2distanceSlow(X)
after = current_time()
t_slow = after - before
print('{:2.0f} ms'.format(t_slow))

print('Running the vectorized version...')
before = current_time()
Dfast = l2distance(X)
after = current_time()
t_fast = after - before
print('{:2.0f} ms'.format(t_fast))

speedup = t_slow / t_fast
print('The two methods should deviate by very little: {:05.6f}'.format(np.linalg.norm(Dfast - Dslow)))
print('but your NumPy code was {:05.2f} times faster!'.format(speedup))

Running the naïve version...
58608 ms
Running the vectorized version...
21 ms
The two methods should deviate by very little: 0.000002
but your NumPy code was 2790.86 times faster!


How much faster is the code now? With this implementation, we should easily be able to compute the distances between <strong>many more</strong> vectors. It should be clear now, even for small datasets, that the for-loop based implementation could take several days or even weeks to perform basic operations that take seconds or minutes with well-written NumPy code.